In [110]:
import ee
import certifi
import os
import requests
import cv2
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub


In [111]:
os.environ['SSL_CERT_FILE'] = certifi.where()
ee.Authenticate()
ee.Initialize()

In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
import cv2
import numpy as np

# Load the pre-trained model from TensorFlow Hub
model = hub.load("https://tfhub.dev/tensorflow/ssd_mobilenet_v2/2")

# Function to load and preprocess images
def load_image(image_path):
    img = cv2.imread(image_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, (300, 300))
    img = np.expand_dims(img, axis=0)
    return img

# Function to run inference
def run_inference(image_path, model):
    img = load_image(image_path)
    results = model(img)
    return results

# Example usage
image_path = '/Users/emilyzhao/STS/Screenshot 2024-07-13 at 4.48.55 PM.png'
results = run_inference(image_path, model)

# Extracting the detection results
detection_boxes = results['detection_boxes'][0].numpy()
detection_scores = results['detection_scores'][0].numpy()
detection_classes = results['detection_classes'][0].numpy()

# Display the results
for i in range(len(detection_boxes)):
    if detection_scores[i] > 0.5:
        print(f"Detected object {i} with score {detection_scores[i]}")

In [ ]:
# Define the area of interest (Long Island, New York)
long_island = ee.Geometry.Polygon(
    [[[-73.935242, 40.496044],
      [-73.935242, 41.052613],
      [-72.533539, 41.052613],
      [-72.533539, 40.496044]]])

# Define the time period
start_date = '2020-09-01'
end_date = '2020-09-30'

In [ ]:
# Get Sentinel-2 image collection for the specified period and location
image_collection = ee.ImageCollection('COPERNICUS/S2') \
    .filterDate(start_date, end_date) \
    .filterBounds(long_island) \
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20))  # Filter images with less than 20% cloud coverage

# Select the first image for demonstration purposes
image = image_collection.first()

In [ ]:
# Get Sentinel-2 image collection for the specified period and location
image_collection = ee.ImageCollection('COPERNICUS/S2') \
    .filterDate(start_date, end_date) \
    .filterBounds(long_island) \
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20))

# Select the bands of interest (e.g., RGB)
bands = ['B4', 'B3', 'B2']

# Function to extract pixel values
def get_pixel_values(image, region, scale=10):
    # Reduce the region and get the pixel values
    return image.select(bands).reduceRegion(
        reducer=ee.Reducer.toList(),
        geometry=region,
        scale=scale,
        bestEffort=True
    ).getInfo()

# Iterate over the image collection and extract pixel values
pixel_values = []
for image in image_collection.toList(image_collection.size()).getInfo():
    img = ee.Image(image['id'])
    pixel_values.append(get_pixel_values(img, long_island))

In [ ]:
# Get Sentinel-2 image collection for the specified period and location
image_collection = ee.ImageCollection('COPERNICUS/S2') \
    .filterDate(start_date, end_date) \
    .filterBounds(long_island) \
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20))

# Select the bands of interest (e.g., RGB)
bands = ['B4', 'B3', 'B2']

# Function to extract pixel values
def get_pixel_values(image, region, scale=10):
    return image.select(bands).reduceRegion(
        reducer=ee.Reducer.toList(),
        geometry=region,
        scale=scale,
        bestEffort=True,
        maxPixels=1e8
    ).getInfo()

# Get a list of images
image_list = image_collection.toList(image_collection.size())

# Extract pixel values from the first image in the list for demonstration
first_image = ee.Image(image_list.get(0))
pixel_values = get_pixel_values(first_image, long_island)

In [ ]:
import numpy as np

def prepare_pixel_data(pixel_values, image_shape=(300, 300)):
    # Ensure pixel_values is a dictionary containing 'B4', 'B3', and 'B2' keys
    red_band = np.array(pixel_values['B4'])
    green_band = np.array(pixel_values['B3'])
    blue_band = np.array(pixel_values['B2'])

    # Stack the bands along the last dimension
    img = np.stack([red_band, green_band, blue_band], axis=-1)

    # Check the shape
    print(f"Extracted image shape: {img.shape}")

    # Normalize pixel values
    img = img / 255.0

    # Ensure the image is resized to the model's input shape
    img_resized = cv2.resize(img, (image_shape[1], image_shape[0]))

    # Expand dimensions to match model input
    img_resized = np.expand_dims(img_resized, axis=0)

    return img_resized

processed_data = prepare_pixel_data(pixel_values, image_shape=(300, 300))